In [1]:
import pandas as pd
from datetime import datetime
import requests
import json
import warnings

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
races = pd.read_csv(path+'data/races.csv')

rounds = races[races.season == 2021]['round'].unique()

In [3]:
# query results API

for i in rounds:

    url = 'http://ergast.com/api/f1/{}/{}/results.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races'][0]['Results']:
        for n in item:
            print(f'{n} : {item[n]}')
        break
    break

number : 44
position : 1
positionText : 1
points : 25
Driver : {'driverId': 'hamilton', 'permanentNumber': '44', 'code': 'HAM', 'url': 'http://en.wikipedia.org/wiki/Lewis_Hamilton', 'givenName': 'Lewis', 'familyName': 'Hamilton', 'dateOfBirth': '1985-01-07', 'nationality': 'British'}
Constructor : {'constructorId': 'mercedes', 'url': 'http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One', 'name': 'Mercedes', 'nationality': 'German'}
grid : 2
laps : 56
status : Finished
Time : {'millis': '5523897', 'time': '1:32:03.897'}
FastestLap : {'rank': '4', 'lap': '44', 'Time': {'time': '1:34.015'}, 'AverageSpeed': {'units': 'kph', 'speed': '207.235'}}


In [155]:
qual = {
    'season': [],
    'round': [],
    'position': [],
    'circuit_id':[],
    'driver': [],
    'q1': [],
    'q2': [],
    'q3': []
}

In [156]:
# query results API

for i in rounds:
    url = 'http://ergast.com/api/f1/{}/{}/qualifying.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        for n in item['QualifyingResults']:
            qual['season'].append(2021)
            qual['round'].append(i)
            qual['position'].append(n['position'])
            qual['driver'].append(n['Driver']['driverId'])
            qual['circuit_id'].append(item['Circuit']['circuitId'])

            try:
                qual['q3'].append(n['Q3'])
            except:
                qual['q3'].append('OUT')

            try:
                qual['q2'].append(n['Q2'])
            except:
                qual['q2'].append('OUT')

            try:
                qual['q1'].append(n['Q1'])
            except:
                qual['q1'].append('DNF')

pd.DataFrame(qual)

,season,round,position,circuit_id,driver,q1,q2,q3
0,2021,1,1,bahrain,max_verstappen,1:30.499,1:30.318,1:28.997
1,2021,1,2,bahrain,hamilton,1:30.617,1:30.085,1:29.385
2,2021,1,3,bahrain,bottas,1:31.200,1:30.186,1:29.586
3,2021,1,4,bahrain,leclerc,1:30.691,1:30.010,1:29.678
4,2021,1,5,bahrain,gasly,1:30.848,1:30.513,1:29.809
...,...,...,...,...,...,...,...,...
434,2021,22,16,yas_marina,latifi,1:24.338,OUT,OUT
435,2021,22,17,yas_marina,russell,1:24.423,OUT,OUT
436,2021,22,18,yas_marina,raikkonen,1:24.779,OUT,OUT
437,2021,22,19,yas_marina,mick_schumacher,1:24.906,OUT,OUT


In [ ]:
# query API

for i in rounds:

    url = 'http://ergast.com/api/f1/{}/{}/constructorStandings.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
        for n in item:
            print(f'{n} : {item[n]}')
        break

In [ ]:
# query API

for i in rounds:

    url = 'http://ergast.com/api/f1/{}/{}/driverStandings.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
        for n in item:
            print(f'{n} : {item[n]}')
        break

In [ ]:
# query API

schedules = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'date': []
        }

for i in rounds:

    url = 'http://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(2021))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        schedules['season'].append(int(item['season']))
        schedules['round'].append(int(item['round']))
        schedules['circuit_id'].append(item['Circuit']['circuitId'])
        schedules['date'].append(datetime.strptime(item['date']+ 'T' +item['time'], '%Y-%m-%dT%H:%M:%SZ'))

schedules = pd.DataFrame(schedules)
schedules.head()